Test for create_daily_share_quantity()
######################################

Notes:
[x] - create_share_quantity_df() produces a df with 'Date' label in index column. -> Ok
[x] - create_daily_share_quantity() removes 'Date' label in index. -> Add
[x] - create_daily_share_quantity() dates are in ascending order (last day first)  -> Make descending
[x] - create_daily_share_quantity() shares are object dtypes  -> Make Integers
[] - Review if its worth changing the date column so its not the index
[] Function requires create_share_quantity_df(). Rename -> create_empty_share_quantity_df()


In [1]:
import unittest
import pandas as pd
from pandas import Timestamp
from scripts import daily_shares as ds


In [2]:
# Setup
date_range1 = pd.date_range(start="2022-12-01", end="2022-12-06")
dates1 = [
    Timestamp('2022-12-01 00:00:00'),
    Timestamp('2022-12-03 00:00:00'),
    Timestamp('2022-12-05 00:00:00'),
    ]
df1 = pd.DataFrame({
    'Ticker': ['VOO', 'VOO', 'VOO'],
    'Yfinance_Ticker': ['VOO.MX', 'VOO.MX', 'VOO.MX'],
    'Trade': ['Buy', 'Buy', 'Sell'],
    'Shares': [10, 5, -3],
    }, index=dates1)
df1 = df1.rename_axis('Date', axis=1)
yftickers = list(df1['Yfinance_Ticker'].unique())
df2 = ds.create_share_quantity_df(yftickers)


In [3]:
df1

Date,Ticker,Yfinance_Ticker,Trade,Shares
2022-12-01,VOO,VOO.MX,Buy,10
2022-12-03,VOO,VOO.MX,Buy,5
2022-12-05,VOO,VOO.MX,Sell,-3


In [4]:
df2

Date,VOO.MX


In [11]:
# Expected
dates_expected2 = pd.date_range('2022-12-01', periods=6)
expected = pd.DataFrame({
    'VOO.MX': [10, 10, 15, 15, 12, 12],
    }, index=dates_expected2)
expected = expected.rename_axis('Date', axis=1)
expected
#print(expected.dtypes)

Date,VOO.MX
2022-12-01,10
2022-12-02,10
2022-12-03,15
2022-12-04,15
2022-12-05,12
2022-12-06,12


In [ ]:
from datetime import datetime

def create_daily_share_quantity(date_range: datetime, trade_history: 'pd', ticker_list: list, df: 'pd') -> 'pd':
    # For loop: Go through every date:
    for date in date_range:
        # Temporary list
        values = []
        # For loop: Go through every ticker:
        for ticker in ticker_list:
            # Filter dataframe by ticker
            filter_df = (
                trade_history[trade_history['Yfinance_Ticker'] == ticker])
            # Filter dataframe up to date
            day_filter_df = filter_df.loc[:date]
            # Sum shares in dataframe for that ticker
            total_shares = day_filter_df['Shares'].sum()
            # Append product to the temporary list
            values.append(total_shares)
        # Create dictionary with tickers and share values for given date
        new_row = dict(zip(ticker_list, values))
        # Create dataframe from dictionary
        new_row_df = pd.DataFrame(new_row, index=[date])
        new_row_df = new_row_df.rename_axis('Date', axis=1)                    # --------------------> Nueva linea
        #new_row_df = new_row_df.convert_dtypes()
        # Merge new row with dataframe from create_share_quantity_df()
        df = pd.concat([new_row_df, df])
    df.sort_index(ascending=True, inplace=True)                     # --------------------> Nueva linea
    df = df.astype(int)
   
    return df

In [12]:

# Call function (de arriba, no importada, poner ds.create_daily_share_quantity para la otra)
actual = ds.create_daily_share_quantity(
    date_range=date_range1,
    trade_history=df1,
    ticker_list=yftickers,
    df=df2)
actual
#print(actual.dtypes)

Date,VOO.MX
2022-12-01,10
2022-12-02,10
2022-12-03,15
2022-12-04,15
2022-12-05,12
2022-12-06,12


In [14]:
pd.testing.assert_frame_equal(actual, expected, check_freq=False)